In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np

# # Useful for plotting graphs
import matplotlib.pyplot as plt
from seaborn import scatterplot

from sklearn.linear_model import LinearRegression

# # Intent for data exploration
from pandas.plotting import scatter_matrix
from seaborn import scatterplot

# # Important in order to create a test set
from sklearn.model_selection import train_test_split

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

from sklearn.preprocessing import OneHotEncoder

# # For feature engineering
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate

from sklearn.metrics import mean_absolute_error

from joblib import dump

from sklearn.model_selection import ShuffleSplit

from sklearn.model_selection import cross_val_score

from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import PolynomialFeatures

from sklearn.model_selection import validation_curve
from sklearn.model_selection import learning_curve

from seaborn import lmplot, stripplot





# Read in and Check the Data
#### I also shuffle the dataset which will ensure randomisation

In [ ]:
df = pd.read_csv('dataset_vehicles.csv')

df = df.sample(frac=1, random_state=2)
df.reset_index(drop=True, inplace=True)

#### Summary Statistics

In [ ]:
df.describe(include="all")

#### Checking all the columns in the dataset

In [ ]:
df.columns

#### This checks to see how many items are in the dataset

In [ ]:
df.index

#### Illustrating the dimensions of the dataframe. 18938 rows, 19 columns. Shape can be used to determine whether we should use K fold or Holdout. In this case it would be Holdout as it is a large dataset. 

In [ ]:
df.shape

#### Showcasing the datatypes. Notice how there is only 1 integer, the rest are strings. It is clear that I need to convert some objects to integers

In [ ]:
df.dtypes

#### Visual representation of the dataframe. Initial reaction indicates that the dataframe needs to be cleaned. Only showing a few rows to minimise leakage

In [ ]:
df.head()

#### Taking a copy of the df incase I make a big error, I still have the original

In [ ]:
df1 = df.copy()

# Data Cleaning
#### This includes converting objects to integers and removing outliers

#### Removing "Rs" from the price. Removing the comma so it can be converted to an integer. Converting to integer

In [ ]:
df1['Price'] = df1['Price'].str.replace('Rs', '')
df1["Price"] = df1["Price"].str.replace(',', '')
df1["Price"] = df1["Price"].astype(int)

#### Renaming Mileage to Mileage_km so I can extract km in the data to make the values an integer. The reason for renaiming is, so when the value is gone, future readers will understand from the title what it is measured in. The user could eaily think it is miles instead of km as some countries use different measurements

In [ ]:
df1 = df1.rename(columns={'Mileage':'Mileage_km'})

#### Now it is time to remove "km: in the values. Removing the comma so it can be converted to an integer. Converting to integer

In [ ]:
df1["Mileage_km"] = df1["Mileage_km"].str.replace('km', '')
df1["Mileage_km"] = df1["Mileage_km"].str.replace(',', '')
df1["Mileage_km"] = df1["Mileage_km"].astype(int)

#### Renaming Capacity to Capacity so I can extract cc in the data to make it an integer. The reason for renaiming is so when the value is gone future readers will understand from the title what it is measured in. In this case it is measured in cc



In [ ]:
df1 = df1.rename(columns={'Capacity':'Capacity_cc'})

#### Removing "cc" from the capacity column. Removing the comma so it can be converted to an integer. Converting to integer

In [ ]:
df1["Capacity_cc"] = df1["Capacity_cc"].str.replace('cc', '')
df1["Capacity_cc"] = df1["Capacity_cc"].str.replace(',', '')
df1["Capacity_cc"] = df1["Capacity_cc"].astype(int)

#### Demonstrating that I have converted the objects/strings that should be integers, into integers

In [ ]:
df1.dtypes

#### Checking to see if my conversions have worked on the data visually. Only showing head to minimise leakage

In [ ]:
df1.head()

#### Check for NaN in the case of the target values. It is False so nothing further needs to be done. 



In [ ]:
df1["Price"].isnull().values.any()

#### Creating lists of features dedicated to their data type to make it easy to search through them and find things like outliers. Excluding certain features as they are irrelevant and I do not want to mindlessly one-hot-encode them. I have excluded certain features in the features list as it increases my error when I one-hot-encode them. Therefore, I only have numeric features in my feature list

In [ ]:
numeric_features = ["Year", "Capacity_cc", "Mileage_km"]
nominal_features = ["Condition", "Transmission", "Fuel"]
features = ["Year", "Capacity_cc", "Mileage_km"]

#### The values, in the case of numerical-valued features. Checking these values to see if there are any outliers. It is clear that there are many outliers here. For capacity, it is impossible for an engine to have a size of 1, 2, 3cc etc. These values need to be removed.

In [ ]:
for feature in numeric_features:
    print(feature, df1[feature].unique())

#### Illustrating how many rows and columns are in the dataframe before I start deleting outliers

In [ ]:
df1.shape

#### Checking to see what certain features look like on graphs and also checking to see where the outliers lie

In [ ]:
plot = stripplot(x="Fuel", y="Capacity_cc", data=df1, jitter=0.2)

#### Delete examples whose Capacity is too small and too big. Selected these numbers from research on the web as to what the engine cc should be and by also looking at outliers on the graph



In [ ]:
df1 = (df1[(df1["Capacity_cc"] >= 300) & (df1["Capacity_cc"] < 7000)]).copy()

In [ ]:
plot = stripplot(x="Condition", y="Price", data=df1, jitter=0.2)

#### Delete examples whose prices are too low and too high. 50000 value was decided through research into Sri Lankin car websites by checking the lowest price of a used car and seeing what the outliers were on the graphs. 158000000 was decided from finding out how much the most expensive car in sri lanka costs. No car exceeds this figure in Sri Lanka







In [ ]:
df1 = (df1[(df1["Price"] > 50000) & (df1['Price'] <= 158000000)]).copy()

#### Illustrating that the examples have been deleted. If you look at the y-axis, it has narrowed down the range. This is why it may look like the "New Condition" chart has grown

In [ ]:
plot = stripplot(x="Condition", y="Price", data=df1, jitter=0.2)

In [ ]:
plot = stripplot(x="Condition", y="Mileage_km", data=df1, jitter=0.2)

#### Thinking of deleting used cars that have extremely low mileage but realised that there is no way of proving that they were not dirven


In [ ]:
df2 = (df1[(df1["Mileage_km"] < 100) & (df1['Condition'] == 'Used')]).copy()

#### From my research, a cars life expectancy is 200,000 km. Taking engine replacemnents into account I have over doubled this figure to eliminate outliers. 



In [ ]:
df1 = (df1[(df1['Mileage_km'] < 500000)]).copy()

#### Reset the index

In [ ]:
df1.reset_index(drop=True, inplace=True)

#### Check shape after deletion

In [ ]:
df1.shape

# Creating a Test Set
#### Creating test set before data exploration so there is no leakage



In [ ]:
dev_df1, test_df1 = train_test_split(df1, train_size=0.8, random_state=2)

# Extract the features but leave as a DataFrame
dev_X = dev_df1[features]
test_X = test_df1[features]

# Target values, converted to a 1D numpy array
dev_y = dev_df1["Price"].values
test_y = test_df1["Price"].values

#### It can be good to do this on a copy of the dataset, excluding the test set 

In [ ]:
copy_df1 = dev_df1.copy()

# Data Exploration

In [ ]:
m = scatter_matrix(copy_df1, figsize=(15, 15))

#### Investigation into the price of vehicles. As expected, Manual cars price is low

In [ ]:
plot = stripplot(x="Transmission", y="Price", data=copy_df1, jitter=0.3)

#### Attempting to do logs on graphs to see if any interesting changes are made. Attempting features like these for feature engineering

In [ ]:
df4 = copy_df1.copy()
df4['Capacity_cc'] = np.log(copy_df1['Capacity_cc'])
plot = scatterplot(x="Capacity_cc", y="Price", data=df4)

#### Illustrating the correlations between the numeric valued columns before feature engineering is performed

In [ ]:
copy_df1.corr()

# Feature Engineering

In [ ]:
# copy_df1['ycmr'] = (copy_df1["Capacity_cc"] + copy_df1["Year"]) - copy_df1["Mileage_km"]
# copy_df1['ycr'] = copy_df1["Year"] + np.log(copy_df1['Capacity_cc'])
# copy_df1['ycr'] = copy_df1["Capacity_cc"]
copy_df1['ycr'] = (copy_df1["Capacity_cc"] + copy_df1["Year"])




#### Plotting my new feature against price to get a visualisation of the correlation

In [ ]:
plot = scatterplot(x="ycr", y="Price", data=copy_df1)

#### Checking the correlations between the standard features and the new features I have created

In [ ]:
copy_df1.corr()

#### Writing a class based on the new features created

In [ ]:
class InsertYCR(BaseEstimator, TransformerMixin):

    def __init__(self, insert=True):
        self.insert = insert
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        if self.insert:
            X['ycr'] = X['Capacity_cc'] + X['Year']  
            X = X.replace( [ np.inf, -np.inf ], np.nan )
        return X


In [ ]:
class TransformerFromHyperP(BaseEstimator, TransformerMixin):

    def __init__(self, transformer=None):
        self.transformer = transformer
        
    def fit(self, X, y=None):
        if self.transformer:
            self.transformer.fit(X, y)
        return self
    
    def transform(self, X, y=None):
        if self.transformer:
            return self.transformer.transform(X)
        else:
            return X

# Preprocessor
#### I discovered that one-hot-enoding suprisingly ended up increasing my prediction error so I left that out

In [ ]:
preprocessor = ColumnTransformer([
        ("num", Pipeline([("ycr", InsertYCR()),
                          ("imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
                          ("scaler", TransformerFromHyperP())]), 
                numeric_features)]
    ,remainder="passthrough")


#         ("nom", Pipeline([("imputer", SimpleImputer(missing_values=np.nan, strategy="most_frequent")), 
#                           ("binarizer", OneHotEncoder(handle_unknown="ignore"))]), 
#                 nominal_features)],

# Ridge Regression

In [ ]:
# Create a pipeline that combines the preprocessor with ridge regression
ridge = Pipeline([
    ("preprocessor", preprocessor),
    ("predictor", Ridge())])

# Create a dictionary of hyperparameters for ridge regression
ridge_param_grid = {"preprocessor__num__ycr__insert": [True, False],
                     "preprocessor__num__scaler__transformer": [StandardScaler(), MinMaxScaler(), RobustScaler()],
                     "predictor__alpha": [80.0, 85.0, 90.0, 95.0, 100.0]}

# Create the grid search object which will find the best hyperparameter values based on validation error
ridge_gs = GridSearchCV(ridge, ridge_param_grid, scoring="neg_mean_absolute_error", cv=10)

# Run grid search by calling fit
ridge_gs.fit(dev_X, dev_y)

# Let's see how well we did
ridge_gs.best_params_, ridge_gs.best_score_

In [ ]:
ridge.set_params(**ridge_gs.best_params_) 
scores = cross_validate(ridge, dev_X, dev_y, cv=10, 
                        scoring="neg_mean_absolute_error", return_train_score=True)
print("Training error: ", np.mean(np.abs(scores["train_score"])))
print("Validation error: ", np.mean(np.abs(scores["test_score"])))

# Error Estimation
#### As you can see, some models are much better than others

In [ ]:
linear_model = Pipeline([
    ("preprocessor", preprocessor),
    ("predictor", LinearRegression())])

In [ ]:
quadratic_model = Pipeline([
    ("poly", PolynomialFeatures(degree=3, include_bias=False)),
    ("predictor", LinearRegression())
])

In [ ]:
poly_model = Pipeline([
    ("poly", PolynomialFeatures(degree=10, include_bias=False)),
    ("predictor", LinearRegression())
])

In [ ]:
# Error estimation for the linear model.
np.mean(cross_val_score(linear_model, dev_X, dev_y, scoring="neg_mean_absolute_error", cv=10))

In [ ]:
# Error estimation for the quadratic model.
np.mean(cross_val_score(quadratic_model, dev_X, dev_y, scoring="neg_mean_absolute_error", cv=10))

In [ ]:
# Error estimation for the poly model.
np.mean(cross_val_score(poly_model, dev_X, dev_y, scoring="neg_mean_absolute_error", cv=10))

# Under and Overfitting

#### Underfitting. Note how high both kinds of error are.

In [ ]:
linear_model = LinearRegression()

scores = cross_validate(linear_model, dev_X, dev_y, cv=10, scoring="neg_mean_absolute_error", return_train_score=True)
print("Training error: ", np.mean(np.abs(scores["train_score"])))
print("Validation error: ", np.mean(np.abs(scores["test_score"])))


#### Overfitting. Polynomial regression with degree 10 overfits our synthetic training set. Note how low training error is, but validation error is high.

In [ ]:
poly_model_d10 = Pipeline([
    ("poly", PolynomialFeatures(degree=10, include_bias=False)),
    ("predictor", LinearRegression())
])

scores = cross_validate(poly_model_d10, dev_X, dev_y, cv=10, scoring="neg_mean_absolute_error", return_train_score=True)
print("Training error: ", np.mean(np.abs(scores["train_score"])))
print("Validation error: ", np.mean(np.abs(scores["test_score"])))

#### Just right. Quadratic model with degree 3 is about right for our synthetic training set. Note how training error is low but not surprisingly so, and validation error is similar.

In [ ]:
quartic_model = Pipeline([
    ("poly", PolynomialFeatures(degree=3, include_bias=False)),
    ("predictor", LinearRegression())
])


scores = cross_validate(quartic_model, dev_X, dev_y, cv=10, scoring="neg_mean_absolute_error", return_train_score=True)
print("Training error: ", np.mean(np.abs(scores["train_score"])))
print("Validation error: ", np.mean(np.abs(scores["test_score"])))

# quartic_model_gs.best_params_, quartic_model_gs.best_score_

## Note
#### When a model is OVERFITTING: it is too complex. You need to reduce the amount of features being used
#### When a model is UNDERFITTING: it is not complex enough. You need to add more features

# KNN
#### This is the most accurate out of all of my prediction models, therefore this is the one I will use. Considering that the best nerest neighbour is not the last one we do not need to extend our range. If the best value was the last neighbour we would need to increase our range of neighbours to maybe 15

In [ ]:
# Create a pipeline that combines the preprocessor with kNN
knn = Pipeline([
    ("preprocessor", preprocessor),
    ("predictor", KNeighborsRegressor())])

# Create a dictionary of hyperparameters for kNN
knn_param_grid = {"predictor__n_neighbors": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                  "preprocessor__num__ycr__insert": [True, False],
                  "preprocessor__num__scaler__transformer": [StandardScaler(), MinMaxScaler(), RobustScaler()]}

# Create the grid search object which will find the best hyperparameter values based on validation error
knn_gs = GridSearchCV(knn, knn_param_grid, scoring="neg_mean_absolute_error", cv=10)

# Run grid search by calling fit
knn_gs.fit(dev_X, dev_y)

# Let's see how well we did
knn_gs.best_params_, knn_gs.best_score_

In [ ]:
scores = cross_validate(knn, dev_X, dev_y, cv=10, scoring="neg_mean_absolute_error", return_train_score=True)
print("Training error: ", np.mean(np.abs(scores["train_score"])))
print("Validation error: ", np.mean(np.abs(scores["test_score"])))

# Evaluate on the test set
#### Considering KNN has the lowest error for me on the training set, I will use this model on the test set.

In [ ]:
knn.set_params(**knn_gs.best_params_) 
knn.fit(dev_X, dev_y)
mean_absolute_error(test_y, knn.predict(test_X))